In [ ]:
import re
import os
import numpy as np
import pandas as pd
from pandas._libs.tslibs import dtypes, timestamps
import subprocess
from functools import reduce
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
import pickle as pkl
# Using matplotlib backend: QtAgg

In [ ]:
def printRow(row):
    print("TIME:", row['system_time'])
    print(row['action'])
    print("==============")

system = "aurora"
if system == "aurora":
    stat_dir = "/home/gsd/andrelucena/statistics"
elif system == "slurm":
    stat_dir = "/projects/a97485/statistics"

model = "resnet50"
n_nodes = "4"
n_epochs = "2"
batch_size = "64"
save_every = "1"
log = "true"

stat_test = "eBPFs_subset_t2"
if system == "aurora":
    test_name = model + "_" + n_epochs + "_" + batch_size + "_" + save_every + "_" + log
elif system == "slurm":
    test_name = model + "_" + n_nodes + "_" + n_epochs + "_" + batch_size + "_" + save_every + "_" + log

full_test_path = stat_dir + "/" + stat_test + "/" + test_name
if system == "slurm":
    full_test_path += "/aurora03"

p = subprocess.Popen(["scp", f"{system}:{full_test_path}/biopattern.bt", "./tmp/file"])
sts = os.waitpid(p.pid, 0)

data = None

with open("./tmp/file", "r") as fd:
    data = re.sub(r"\^C", "", fd.read())
with open("./tmp/file", "w") as fd:
    fd.write(data)

df = pd.read_csv("./tmp/file", skiprows=[0,1,2,3,4,5,6], index_col=False, delimiter=" +")

df['TIME'] = pd.to_datetime(df['TIME'])

os.remove("./tmp/file")

df.head()

In [ ]:
def plotAction(df_param: pd.DataFrame, x, y, title):

    dt = df_param

    # plots action with time
    fig = plt.figure()
    plt.plot(dt[x], dt[y], markersize = 5.0, marker = "+")
    plt.xlabel(x)
    plt.ylabel(y)
    plt.title(title)
    # plt.legend()

    return fig

In [ ]:
fig = plotAction(df, 'TIME', '%RND', '')

with open("fig.pkl", "bw+") as fd:
    pkl.dump(fig, fd)


In [ ]:
with open("fig.pkl", "br") as fd:
    fig = pkl.load(fd)

fig.show()

In [ ]:
plotAction(df, 'TIME', '%SEQ', '')

In [ ]:
plotAction(df, 'TIME', 'COUNT', '')

In [ ]:
plotAction(df, 'TIME', 'KBYTES', '')